## ROI analysis of the NF task 

This code performs a ROI analysis on the contrast maps regions of the Sense of Agency (SoA) network of n = 18 participants. The con values are taken from the gamephase > rest contrast for each of the 9 runs of the NF training. 
It uses Lmer from the pymer4 package (Jolly, (2018). Pymer4: Connecting R and Python for Linear Mixed Modeling. Journal of Open Source Software, 3(31), 862, https://doi.org/10.21105/joss.00862. )

In [ ]:
import numpy as np
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import sys
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
import statsmodels.api as sm
import statsmodels as sts
import statsmodels.stats.multitest as smm
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf
import seaborn as sns
from statsmodels.stats.contrast import ContrastResults
from statsmodels.stats.outliers_influence import summary_table
import patsy
from patsy import build_design_matrices, dmatrix
from pymer4.models import Lmer,Lm
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from matplotlib.ticker import MaxNLocator
emmeans = importr('emmeans')


In [ ]:
# Read in mean BOLD signal per run for ROIs of the SoA network (rTPJ, lTPJ, bilateral SMA, bilateral DLPFC, bilateral IC)


# Right TPJ (NF training task)
file_path_rTPJ = './ROI_rTPJ.xlsx'

# Read the sheet with mean BOLD signal per run
rTPJ_df = pd.read_excel(file_path_rTPJ, sheet_name='run_mean_values')

# Leave out last column (Run 9 - Run 1 difference column)
rTPJ_df = rTPJ_df.iloc[:,0:10]

# Preprocess the dataframe
rTPJ_df.rename(columns={rTPJ_df.columns[0]: 'P-Code'}, inplace=True)
rTPJ_df.set_index('P-Code', inplace=True)


# Left TPJ (NF training task) 
# Excel file with mean BOLD signal per run per subject in the lTPJ ROI
file_path_lTPJ = './ROI_lTPJ.xlsx'

lTPJ_df = pd.read_excel(file_path_lTPJ)

lTPJ_df.set_index('P-Code', inplace=True)





In [ ]:
# Read the sheet with mean contrast values for each region of the atlas and preprocess the dataframe

file_path = './Glasser_atlas_ROIs.xlsx'

# Read all runs into a dictionary instead of a list
runs = {f'Run{i}': pd.read_excel(file_path, sheet_name=f'Run{i}').set_index('P-Code') 
        for i in range(1, 10)}

# Define ROIs of interest
rois = {
    'left_insula': ['ROI_106', 'ROI_167'],
    'right_insula': ['ROI_283', 'ROI_347'],
    'left_SMA': ['ROI_ 44', 'ROI_ 55'],
    'right_SMA': ['ROI_224','ROI_235'],
    'left_DLPFC': ['ROI_ 83', 'ROI_ 84', 'ROI_ 86'],
    'right_DLPFC': ['ROI_263', 'ROI_264', 'ROI_266'],
}

# Compute mean values for each ROI and run in a single nested dictionary comprehension
roi_results = {
    roi_name: pd.concat(
        [(run[roi_cols].mean(axis=1) if len(roi_cols) > 1 else run[roi_cols[0]]) 
         for run in runs.values()], 
        axis=1
    ).set_axis(runs.keys(), axis=1)
    for roi_name, roi_cols in rois.items()
}

# Optional: assign each ROI to a variable
left_insula_df = roi_results['left_insula']
right_insula_df = roi_results['right_insula']
left_SMA_df = roi_results['left_SMA']
right_SMA_df = roi_results['right_SMA']
left_DLPFC_df = roi_results['left_DLPFC']
right_DLPFC_df = roi_results['right_DLPFC']

# Combine hemispheres for each region
insula_df = (left_insula_df + right_insula_df)/2
SMA_df = (left_SMA_df + right_SMA_df)/2
DLPFC_df = (left_DLPFC_df + right_DLPFC_df)/2
# TPJ_df = (lTPJ_df + rTPJ_df)/2  # Make sure lTPJ_df and rTPJ_df are defined


In [ ]:
# Export dfs as excel files
output_path = "./NF_SoA_network.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    rTPJ_df.to_excel(writer, sheet_name='rTPJ', index=True)
    lTPJ_df.to_excel(writer, sheet_name='lTPJ', index=True)
    SMA_df.to_excel(writer, sheet_name='SMA', index=True)
    insula_df.to_excel(writer, sheet_name='Insula', index=True)
    DLPFC_df.to_excel(writer, sheet_name='DLPFC', index=True)

In [ ]:
# Generate a linear plot for each ROI

# Function to plot the data
def plot_roi(roi_df, roi_name):
    """ Plot the mean contrast estimates for a given ROI.
    Args:
        roi_df (DataFrame): DataFrame containing the contrast estimates for the ROI.
        roi_name (str): Name of the ROI.
    """ 
    # Create a mask for responders and non-responders
    roi_df['subject_id'] = range(18)
    roi_df.set_index('subject_id', inplace=True)
    
    responder_mask = np.array([False, False, False, True, False, False, True, False, False, True, True, True, True, False, True, True, False, False])
    nonresponder_mask = ~responder_mask
    #nonresponder_mask = np.array([True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True])

    # Separate learners and non-learners data 
    learn_roi = roi_df[responder_mask]
    non_learn_roi = roi_df[nonresponder_mask]

    # Split data by sessions for learners
    learners_session_means = []
    learners_session_sems = []
    non_learners_session_means = []
    non_learners_session_sems = []

    # Define session indices
    sessions = [(0, 3), (3, 6), (6, 9)]  # Column index ranges for each session
    for start, end in sessions:
        # Learners
        session_data = learn_roi.iloc[:, start:end]
        session_mean = session_data.mean(axis=0)  # Mean across subjects
        session_sem = session_data.sem(axis=0)    # Standard error of mean across subjects
        learners_session_means.append(session_mean)
        learners_session_sems.append(session_sem)
        
        # Non-learners
        session_data_nonlearn = non_learn_roi.iloc[:, start:end]
        session_mean_nonlearn = session_data_nonlearn.mean(axis=0)
        session_sem_nonlearn = session_data_nonlearn.sem(axis=0)
        non_learners_session_means.append(session_mean_nonlearn)
        non_learners_session_sems.append(session_sem_nonlearn)

    # Flatten session data for plotting
    learners_means = np.concatenate(learners_session_means)
    learners_sems = np.concatenate(learners_session_sems)
    nonlearners_means = np.concatenate(non_learners_session_means)
    nonlearners_sems = np.concatenate(non_learners_session_sems)

    # X-axis: Runs (1-3 for each session, making it 1-9 across three sessions)
    runs = np.arange(1, 10)

    # Plotting
    fig, ax = plt.subplots(dpi = 300, figsize=(10, 6))

    # Plot mean with error bands for Learners
    ax.plot(runs, learners_means, label='Responders (n=8)', color='#1E5285', marker='o', markersize = 6, linestyle = '--')
    ax.fill_between(runs, learners_means - learners_sems, learners_means + learners_sems, color='#1E5285', alpha=0.2)

    # Plot mean with error bands for Non-Learners
    ax.plot(runs, nonlearners_means, label='Non-responders (n=10)', color='#ADADAD', marker='o')
    ax.fill_between(runs, nonlearners_means - nonlearners_sems, nonlearners_means + nonlearners_sems, color='#ADADAD', alpha=0.2)

    # Customize the plot
    ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9])
    ax.set_xticklabels(['1', '2', '3', '4', '5', '6', '7', '8', '9'], size = 16)
    ax.set_xlabel('Run', size=18)
    ax.set_ylabel('Contrast estimate (game > rest) ', size=18)
    ax.set_ylim(0, 0.6)
    # ax.set_ylim(-0.1, 0.25) #(for BOLD signal contrast)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_title(roi_name, size=16, pad = 15)
    # Move legend slightly higher in the center right
    ax.legend(loc='center right', bbox_to_anchor=(1, 0.7))

    # Adding session dividers for clarity
    for session_divider in [3.5, 6.5]:
        ax.axvline(session_divider, color='gray', linestyle='--', alpha=0.5)
    ax.text(2, ax.get_ylim()[1]*0.95, 'Session 1', ha='center', size=12)
    ax.text(5, ax.get_ylim()[1]*0.95, 'Session 2', ha='center', size=12)
    ax.text(8, ax.get_ylim()[1]*0.95, 'Session 3', ha='center', size=12)

    plt.tight_layout()
    plt.show()


# Plot for desired ROI, e.g. left insula
plot_roi(left_insula_df, 'left IC')


In [ ]:
# Function to run the linear mixed model and plot the fitted values (estimated marginal means with CIs)
def run_lmm(roi_df, roi_name, y_lim):
    """ Run a linear mixed model on the given ROI DataFrame.
    Args:
        roi_df (DataFrame): DataFrame containing the contrast estimates for the ROI.
        roi_name (str): Name of the ROI.
    Returns:  LLM output & plot with fitted results
    """

    # Prepare the dataframe and add the covariates
    roi_df['subject_id'] = range(18)
    roi_df.set_index('subject_id', inplace=True)
    roi_df['Subject'] = np.arange(1, 19)

    mean_BOLD_long = roi_df.copy()
    # add covariates (age, sex, BDI etc.)
    # mean_BOLD_long['Age'] = ..
    
    # define responders(learners)/ non-responders(non-learners)
    #mean_BOLD_long['Group'] = ['Non-learner', 'Non-learner', ...] 
                           

    df = mean_BOLD_long.reset_index()

    # Melt the DataFrame, transforming each "Run" column into rows under "Run" and "BOLD_Value" columns
    df_long = pd.melt(df,
                    id_vars=['subject_id', 'Subject', 'Age', 'BDISTAI', 'Sex', 'Group'],
                    value_vars=['Run1', 'Run2', 'Run3', 'Run4', 'Run5','Run6', 'Run7', 'Run8', 'Run9'],
                    var_name='Run',
                    value_name='BOLD_Value')

    # Clean up the "Run" column to have numeric values only
    df_long['Run'] = df_long['Run'].str.extract(r'(\d+)').astype(int)

    # Drop the subject_id column
    df_long = df_long.drop(columns=['subject_id'])

    # Total rows for each session block (3 * 18)
    session_size = 3 * 18

    # Calculate the total number of rows in df_long
    total_rows = df_long.shape[0]

    # Create the session array repeating 1, 2, 3 for each block of 3*18 rows
    df_long['Session'] = np.repeat([1, 2, 3], session_size)

    # Make sure 'Group', 'Run', and 'Session' are treated as categorical variables
    df_long['Group'] = df_long['Group'].astype('category')
    df_long['Run'] = df_long['Run'].astype('category')
    df_long['Session'] = df_long['Session'].astype('category')

    # Define the model formula
    formula = 'BOLD_Value ~ Group * Run + BDISTAI + Age + Sex'

    # Set up the mixed model
    model = smf.mixedlm(formula, df_long, groups=df_long['Subject'], re_formula="~1")
    result = model.fit(method=['powell','lbfgs'])


    # --- Marginal means and CIs ---
    # Prepare prediction grid
    predict_df = df_long[['Group', 'Run']].drop_duplicates().reset_index(drop=True)
    predict_df['BDISTAI'] = df_long['BDISTAI'].mean()

    predict_df['Age'] = df_long['Age'].mean()
    predict_df['Sex'] = df_long['Sex'].mode()[0]

    # Ensure categorical consistency
    predict_df['Group'] = predict_df['Group'].astype(df_long['Group'].dtype)
    predict_df['Run'] = predict_df['Run'].astype(df_long['Run'].dtype)

    # Add dummy BOLD_Value for patsy
    predict_df['BOLD_Value'] = 0

    # Get model predictions
    predict_df['predicted'] = result.predict(predict_df)

    # Compute design matrix for fixed effects
    # (the [1] is for the RHS, i.e., the predictors)
    X_new = patsy.dmatrix(result.model.data.design_info.builder, predict_df, return_type='dataframe')

    # Get fixed effects covariance matrix
    fe_cov = result.cov_params().loc[result.fe_params.index, result.fe_params.index]

    # Calculate standard errors (for each row: x^T Cov x)
    se = np.sqrt(np.einsum('ij,jk,ik->i', X_new.values, fe_cov.values, X_new.values))
    predict_df['se'] = se
    predict_df['ci_lower'] = predict_df['predicted'] - 1.96 * se
    predict_df['ci_upper'] = predict_df['predicted'] + 1.96 * se

    # --- Plot ---
    plt.figure(figsize=(8.5, 6), dpi = 300)

    # Customize the plot and add error bands
    colors = {'Learner': '#6AA098', 'Non-learner': '#ADADAD'}
    linestyles = {'Learner': '--', 'Non-learner': '-'}
    markers = {'Learner': 'o', 'Non-learner': 'o'}

    # Plot lines for each group
    for group, label in zip(['Learner', 'Non-learner'], ['Responders', 'Non-Responders']):
        group_df = predict_df[predict_df['Group'] == group].sort_values('Run')
        plt.plot(
            group_df['Run'].astype(int),
            group_df['predicted'],
            label=label,  # Custom label for legend
            color=colors.get(group, 'gray'),
            linestyle=linestyles.get(group, '-'),
            marker=markers.get(group, 'o')
        )
        plt.fill_between(
            group_df['Run'].astype(int),
            group_df['ci_lower'],
            group_df['ci_upper'],
            alpha=0.2,
            color=colors.get(group, 'gray'),
        )

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # Remove duplicates
    plt.legend(by_label.values(), by_label.keys(), loc= 'upper right', fontsize = 16)

        # Add session dividers
    for x_pos, session_label in zip([3.5, 6.5], ['Session 2', 'Session 3']):
        plt.axvline(x_pos, color='gray', linestyle='--', alpha=0.5)

    # Session titles across runs
    plt.ylim(y_lim)
    ylim = plt.ylim()

    plt.title("dlPFC", size = 30, pad = 10)
    plt.ylabel('Contrast estimate', size = 20)
    plt.xticks (size = 18)
    plt.yticks (size =18)
    plt.xlabel('Run', size = 20)

    plt.gca().yaxis.set_major_locator(MaxNLocator(nbins=6))
    plt.tight_layout()
    plt.show()

    return result


# Apply function to desired ROI, e.g., DLPFC

result_DLPFC = run_lmm(DLPFC_df, 'DLPFC',(-0.1, 0.25))
print(result_DLPFC.summary())